<a href="https://colab.research.google.com/github/waynelee9511cloud/my-colab-notebooks/blob/dm-work/ePRO%E5%88%86%E6%9E%90%E7%A8%8B%E5%BC%8F_v3_2_%E6%8E%92%E5%88%97%E5%A5%BDexcel%E5%B7%A5%E4%BD%9C%E8%A1%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 安裝必要套件
!pip install pandas openpyxl -q
print('✅ 套件安裝完成！')

In [ ]:
#@title 執行ePRO 帳號管理監控程式 - v3.2 (含問卷填寫分析及狀態特殊規則)

# 執行下方程式碼後，系統會要求您上傳檔案：
# - crf_data.xlsx（需包含 SF、DS、SV 工作表）
# - epro_data.xlsx（需包含 epro 工作表）
# - subject_list.xlsx（需包含 Screening No. 和 Status 欄位）
# - epro_content_data.xlsx（需包含 BGTRT1、BGTRT2、NRS1、NRS2 工作表）
# 完成後會自動產生並下載 ePRO分析結果_完整報告.xlsx

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ==================== 設定 ====================
TODAY = pd.Timestamp('2025-11-10')
DELAY_THRESHOLD = 7  # 延遲閾值（天數）
CUTOFF_DAYS = 30  # V1/V2後超過多少天需要檢查

# ==================== 排除名單 ====================
# 如果有需要排除的受試者，請在下方列表中新增 Screening Number
EXCLUDE_SUBJECTS = [
    'ST03008',  # 誤操作，不納入分析
    # 可在此新增更多要排除的受試者 Screening Number
]

print("="*80)
print("🏥 臨床試驗 ePRO 帳號管理監控程式（Colab 版本 v3.2 - 含問卷分析及狀態特殊規則）")
print("="*80)
print(f"\n📅 分析基準日期：{TODAY.strftime('%Y-%m-%d')}")
print(f"⚠️  延遲閾值：超過 {DELAY_THRESHOLD} 天")
print(f"⚠️  V1/V2檢查閾值：超過 {CUTOFF_DAYS} 天")
if EXCLUDE_SUBJECTS:
    print(f"🚫 排除受試者：{', '.join(EXCLUDE_SUBJECTS)}")
print("\n開始執行...\n")


def read_epro_content_data(content_file):
    """讀取 ePRO 問卷填寫內容數據"""
    print("="*80)
    print("📁 步驟 0：讀取 ePRO 問卷填寫內容")
    print("="*80)

    # 讀取四個工作表
    bgtrt1 = pd.read_excel(content_file, sheet_name='BGTRT1')
    bgtrt2 = pd.read_excel(content_file, sheet_name='BGTRT2')
    nrs1 = pd.read_excel(content_file, sheet_name='NRS1')
    nrs2 = pd.read_excel(content_file, sheet_name='NRS2')

    print(f"✓ BGTRT1：{len(bgtrt1)} 筆")
    print(f"✓ BGTRT2：{len(bgtrt2)} 筆")
    print(f"✓ NRS1：{len(nrs1)} 筆")
    print(f"✓ NRS2：{len(nrs2)} 筆")

    # 合併 Background Treatment 資料（BGTRT1 + BGTRT2）
    bgtrt_all = pd.concat([bgtrt1, bgtrt2], ignore_index=True)
    bgtrt_all['Date of record'] = pd.to_datetime(bgtrt_all['Date of record'], errors='coerce').dt.date
    bgtrt_all = bgtrt_all[bgtrt_all['Date of record'].notna()]

    # 合併 NRS 資料（NRS1 + NRS2）
    nrs_all = pd.concat([nrs1, nrs2], ignore_index=True)
    nrs_all['Date of record'] = pd.to_datetime(nrs_all['Date of record'], errors='coerce').dt.date
    nrs_all = nrs_all[nrs_all['Date of record'].notna()]

    print(f"\n✓ 合併後 Background Treatment：{len(bgtrt_all)} 筆有效記錄")
    print(f"✓ 合併後 NRS：{len(nrs_all)} 筆有效記錄")

    # 整理每個受試者的填寫日期
    def summarize_questionnaire_dates(df, questionnaire_name):
        """整理問卷填寫日期摘要"""
        summary = {}

        for sn in df['Screening Number'].unique():
            sn_data = df[df['Screening Number'] == sn].copy()
            dates = sorted(sn_data['Date of record'].unique())

            if len(dates) > 0:
                start_date = dates[0]
                end_date = dates[-1]

                # 計算應該要有的所有日期
                all_dates = pd.date_range(start=start_date, end=end_date, freq='D').date
                filled_dates = set(dates)
                missing_dates = sorted([d for d in all_dates if d not in filled_dates])

                summary[sn] = {
                    'questionnaire': questionnaire_name,
                    'start_date': start_date,
                    'end_date': end_date,
                    'filled_dates': dates,
                    'filled_count': len(dates),
                    'missing_dates': missing_dates,
                    'missing_count': len(missing_dates)
                }

        return summary

    bgtrt_summary = summarize_questionnaire_dates(bgtrt_all, 'Background Treatment')
    nrs_summary = summarize_questionnaire_dates(nrs_all, 'NRS')

    print(f"\n✓ Background Treatment 問卷：{len(bgtrt_summary)} 位受試者有填寫記錄")
    print(f"✓ NRS 問卷：{len(nrs_summary)} 位受試者有填寫記錄")

    return bgtrt_all, nrs_all, bgtrt_summary, nrs_summary


def analyze_questionnaire_compliance(bgtrt_summary, nrs_summary, subject_list, sf, sv):
    """分析問卷填寫合規性"""
    print("\n" + "="*80)
    print("📊 步驟 0.5：分析問卷填寫合規性")
    print("="*80)

    # 準備 subject_list 資料
    subject_list = subject_list.rename(columns={'Screening No.': 'Screening Number'})
    subject_list['First visit date'] = pd.to_datetime(subject_list['First visit date'], errors='coerce').dt.date
    subject_list['Last visit date'] = pd.to_datetime(subject_list['Last visit date'], errors='coerce').dt.date

    # 分析結果
    compliance_results = []

    for _, row in subject_list.iterrows():
        sn = row['Screening Number']
        first_visit = row['First visit date']
        last_visit = row['Last visit']
        last_visit_date = row['Last visit date']
        status = row.get('Status', '')  # 取得 Status 欄位

        if pd.isna(first_visit):
            continue

        # 決定應該填寫問卷的日期區間
        expected_end_date = None

        # 規則1: Screening Failure 狀態的特殊處理
        if status == 'Screening Failure':
            # 嘗試從 SF 工作表取得 Date of screening failure
            sf_date = None
            if not sf.empty and 'Date of screening failure' in sf.columns:
                sf_record = sf[sf['Screening Number'] == sn]
                if not sf_record.empty:
                    sf_date = sf_record.iloc[0]['Date of screening failure']

            if pd.notna(sf_date):
                # 有 Date of screening failure，使用該日期
                expected_end_date = sf_date
            else:
                # 沒有 Date of screening failure，取 V1(D-28 to -1) 的 Visit Date + 27天
                if not sv.empty and 'Visit Date' in sv.columns and 'VISIT' in sv.columns:
                    v1_record = sv[(sv['Screening Number'] == sn) & (sv['VISIT'] == 'V1(D-28 to -1)')]
                    if not v1_record.empty:
                        v1_date = v1_record.iloc[0]['Visit Date']
                        if pd.notna(v1_date):
                            expected_end_date = v1_date + timedelta(days=27)

        # 規則2: Screening 狀態的特殊處理
        elif status == 'Screening':
            # 計算 First visit date 距離今天的天數
            days_since_first_visit = (TODAY.date() - first_visit).days

            if days_since_first_visit > 28:
                # 超過28天，使用 First visit date + 27天
                expected_end_date = first_visit + timedelta(days=27)
            else:
                # 沒有超過28天，使用今天
                expected_end_date = TODAY.date()

        # 原有規則: 其他狀態的處理
        if expected_end_date is None:
            if pd.notna(last_visit) and 'V9' in str(last_visit).upper() or 'EOS' in str(last_visit).upper():
                # Last visit 是 V9/EOS，用 Last visit date
                if pd.notna(last_visit_date):
                    expected_end_date = last_visit_date
                else:
                    expected_end_date = TODAY.date()
            else:
                # Last visit 不是 V9/EOS，用 TODAY
                expected_end_date = TODAY.date()

        expected_start_date = first_visit

        # 計算應該填寫的所有日期
        expected_dates = set(pd.date_range(start=expected_start_date, end=expected_end_date, freq='D').date)

        # 檢查 Background Treatment
        bgtrt_info = bgtrt_summary.get(sn, {})
        if bgtrt_info:
            bgtrt_filled = set(bgtrt_info['filled_dates'])
            bgtrt_missing = expected_dates - bgtrt_filled
            bgtrt_extra = bgtrt_filled - expected_dates
        else:
            bgtrt_filled = set()
            bgtrt_missing = expected_dates
            bgtrt_extra = set()

        # 檢查 NRS
        nrs_info = nrs_summary.get(sn, {})
        if nrs_info:
            nrs_filled = set(nrs_info['filled_dates'])
            nrs_missing = expected_dates - nrs_filled
            nrs_extra = nrs_filled - expected_dates
        else:
            nrs_filled = set()
            nrs_missing = expected_dates
            nrs_extra = set()

        # 記錄結果
        result = {
            'Screening Number': sn,
            'First visit date': expected_start_date,
            'Last visit': last_visit,
            'Last visit date': last_visit_date,
            'Expected end date': expected_end_date,
            'Expected days': len(expected_dates),

            # Background Treatment
            'BGTRT filled days': len(bgtrt_filled),
            'BGTRT missing days': len(bgtrt_missing),
            'BGTRT missing dates': sorted(list(bgtrt_missing)) if bgtrt_missing else [],
            'BGTRT extra days': len(bgtrt_extra),
            'BGTRT extra dates': sorted(list(bgtrt_extra)) if bgtrt_extra else [],
            'BGTRT compliance': f"{len(bgtrt_filled)/len(expected_dates)*100:.1f}%" if expected_dates else "N/A",

            # NRS
            'NRS filled days': len(nrs_filled),
            'NRS missing days': len(nrs_missing),
            'NRS missing dates': sorted(list(nrs_missing)) if nrs_missing else [],
            'NRS extra days': len(nrs_extra),
            'NRS extra dates': sorted(list(nrs_extra)) if nrs_extra else [],
            'NRS compliance': f"{len(nrs_filled)/len(expected_dates)*100:.1f}%" if expected_dates else "N/A"
        }

        compliance_results.append(result)

    compliance_df = pd.DataFrame(compliance_results)

    # 統計
    print(f"\n✓ 分析完成：{len(compliance_df)} 位受試者")

    # 找出有問題的受試者
    bgtrt_issues = compliance_df[compliance_df['BGTRT missing days'] > 0]
    nrs_issues = compliance_df[compliance_df['NRS missing days'] > 0]
    bgtrt_extra_issues = compliance_df[compliance_df['BGTRT extra days'] > 0]
    nrs_extra_issues = compliance_df[compliance_df['NRS extra days'] > 0]

    print(f"\n📊 問卷填寫情況統計：")
    print(f"   Background Treatment 有缺漏：{len(bgtrt_issues)} 位")
    print(f"   NRS 有缺漏：{len(nrs_issues)} 位")
    print(f"   Background Treatment 有多填：{len(bgtrt_extra_issues)} 位")
    print(f"   NRS 有多填：{len(nrs_extra_issues)} 位")

    return compliance_df, bgtrt_issues, nrs_issues, bgtrt_extra_issues, nrs_extra_issues


def read_crf_data(crf_file):
    """讀取 CRF 數據"""
    print("\n" + "="*80)
    print("📁 步驟 1：讀取 CRF 數據")
    print("="*80)

    # 讀取 SF (Screening Failure)
    sf = pd.DataFrame()
    if 'SF' in pd.ExcelFile(crf_file).sheet_names:
        sf = pd.read_excel(crf_file, sheet_name='SF')
        if 'Date of screening failure' in sf.columns:
            sf['Date of screening failure'] = pd.to_datetime(sf['Date of screening failure'], errors='coerce').dt.date
        print(f"✓ SF 工作表：{len(sf)} 筆")

    # 讀取 DS (Disposition)
    ds = pd.DataFrame()
    if 'DS' in pd.ExcelFile(crf_file).sheet_names:
        ds = pd.read_excel(crf_file, sheet_name='DS')
        if 'Date of completion/ Early Termination/ Withdrawal' in ds.columns:
            ds['Date of completion/ Early Termination/ Withdrawal'] = pd.to_datetime(
                ds['Date of completion/ Early Termination/ Withdrawal'], errors='coerce'
            ).dt.date
        print(f"✓ DS 工作表：{len(ds)} 筆")

    # 讀取 SV (Study Visits)
    sv = pd.DataFrame()
    if 'SV' in pd.ExcelFile(crf_file).sheet_names:
        sv = pd.read_excel(crf_file, sheet_name='SV')
        if 'Visit Date' in sv.columns:
            sv['Visit Date'] = pd.to_datetime(sv['Visit Date'], errors='coerce').dt.date
        print(f"✓ SV 工作表：{len(sv)} 筆")

    return sf, ds, sv


def read_epro_data(epro_file):
    """讀取 ePRO 數據"""
    print("\n" + "="*80)
    print("📁 步驟 2：讀取 ePRO 數據")
    print("="*80)

    epro = pd.read_excel(epro_file, sheet_name='epro')
    print(f"✓ 原始數據：{len(epro)} 筆")
    print(f"   原始欄位：{epro.columns.tolist()}")

    # 建立欄位映射字典（支援多種可能的欄位名稱）
    column_mapping = {}

    # Subject/Screening Number 欄位
    for col in epro.columns:
        if col.strip().lower() in ['subject', 'screening number', 'screening no.', 'screening no']:
            column_mapping[col] = 'Screening Number'
            break

    # Date/Time 欄位
    for col in epro.columns:
        col_lower = col.lower().strip()
        if ('date' in col_lower and 'time' in col_lower) or col_lower == 'date/time':
            column_mapping[col] = 'epro_date_time'
            break

    # 檢查是否找到必要的欄位
    if 'Screening Number' not in column_mapping.values():
        raise ValueError(f"❌ 找不到 Subject 或 Screening Number 欄位。可用欄位：{epro.columns.tolist()}")
    if 'epro_date_time' not in column_mapping.values():
        raise ValueError(f"❌ 找不到 Date/Time 欄位。可用欄位：{epro.columns.tolist()}")

    # 重新命名欄位
    epro = epro.rename(columns=column_mapping)
    print(f"   重新命名後欄位：{epro.columns.tolist()}")

    # 驗證重新命名成功
    if 'epro_date_time' not in epro.columns or 'Screening Number' not in epro.columns:
        raise ValueError(f"❌ 重新命名失敗！當前欄位：{epro.columns.tolist()}")

    # 清理時間格式
    def clean_datetime(dt_str):
        if pd.isna(dt_str):
            return None
        try:
            dt_str = str(dt_str).split('(')[0].strip()
            dt = pd.to_datetime(dt_str)
            return dt.date()  # 只保留日期
        except:
            return None

    epro['epro_date'] = epro['epro_date_time'].apply(clean_datetime)

    # 注意：原始數據中Status可能拼寫為"Nomal"
    epro['Status'] = epro['Status'].str.strip()
    epro.loc[epro['Status'] == 'Nomal', 'Status'] = 'Normal'

    # 最新狀態（Normal或Expired）
    epro_latest = epro.sort_values('epro_date_time', ascending=False).groupby('Screening Number').first().reset_index()
    print(f"✓ 最新狀態：{len(epro_latest)} 位受試者")
    print(f"   - Normal: {len(epro_latest[epro_latest['Status'] == 'Normal'])}")
    print(f"   - Expired: {len(epro_latest[epro_latest['Status'] == 'Expired'])}")

    # Expired 日期
    epro_expired = epro[epro['Status'] == 'Expired'][['Screening Number', 'epro_date']].copy()
    epro_expired = epro_expired.rename(columns={'epro_date': 'epro_expired_date'})
    print(f"✓ Expired 記錄：{len(epro_expired)} 筆")

    # Normal 日期（開帳日期）- 修正：取第一個 Normal 狀態的日期
    epro_normal = epro[epro['Status'] == 'Normal'].sort_values('epro_date_time').groupby('Screening Number').first().reset_index()
    epro_normal = epro_normal[['Screening Number', 'epro_date']].copy()
    epro_normal = epro_normal.rename(columns={'epro_date': 'epro_normal_date'})
    print(f"✓ Normal 記錄（第一次開帳）：{len(epro_normal)} 筆")

    return epro_latest, epro_expired, epro_normal


def process_sv_data(sv):
    """處理SV數據，提取V1和最後visit"""
    print("\n" + "="*80)
    print("📁 步驟 3：處理 Visit 數據")
    print("="*80)

    if sv.empty:
        print("⚠️ 沒有SV數據")
        return pd.DataFrame(), pd.DataFrame()

    # 提取V1日期
    v1 = sv[sv['VISIT'].str.contains('V1', case=False, na=False)].copy()
    v1 = v1.sort_values('Visit Date').groupby('Screening Number').first().reset_index()
    v1 = v1[['Screening Number', 'Visit Date']].rename(columns={'Visit Date': 'V1_date'})
    print(f"✓ V1 記錄：{len(v1)} 位受試者")

    # 找出每個受試者的最後visit
    sv_sorted = sv.sort_values(['Screening Number', 'Visit Date'])
    last_visit = sv_sorted.groupby('Screening Number').last().reset_index()
    last_visit = last_visit[['Screening Number', 'VISIT', 'Visit Date']].rename(columns={
        'VISIT': 'last_visit',
        'Visit Date': 'last_visit_date'
    })
    print(f"✓ 最後 Visit 記錄：{len(last_visit)} 位受試者")

    return v1, last_visit


def read_subject_list(subject_list_file):
    """讀取 Subject List（試驗狀態）"""
    print("\n" + "="*80)
    print("📁 步驟 3.5：讀取 Subject List（試驗狀態）")
    print("="*80)

    try:
        subject_list = pd.read_excel(subject_list_file)

        # 選取需要的欄位，並重新命名以統一
        if 'Screening No.' in subject_list.columns and 'Status' in subject_list.columns:
            status_data = subject_list[['Screening No.', 'Status']].copy()
            status_data = status_data.rename(columns={
                'Screening No.': 'Screening Number',
                'Status': '試驗狀態'
            })
            print(f"✓ Subject List：{len(status_data)} 筆")
            print(f"   試驗狀態分布：")
            for status, count in status_data['試驗狀態'].value_counts().head(5).items():
                print(f"      - {status}: {count}")
            return status_data, subject_list
        else:
            print("⚠️ Subject List 缺少必要欄位")
            return pd.DataFrame(columns=['Screening Number', '試驗狀態']), subject_list
    except Exception as e:
        print(f"⚠️ 無法讀取 Subject List：{e}")
        return pd.DataFrame(columns=['Screening Number', '試驗狀態']), pd.DataFrame()


def merge_all_data(sf, ds, sv, epro_latest, epro_expired, epro_normal, subject_status):
    """合併所有數據"""
    print("\n" + "="*80)
    print("🔗 步驟 4：合併數據")
    print("="*80)

    # 處理SF
    sf_data = sf[['Screening Number', 'Date of screening failure']].copy() if not sf.empty else pd.DataFrame(columns=['Screening Number', 'Date of screening failure'])
    sf_data['exit_reason'] = 'SF'
    sf_data = sf_data.rename(columns={'Date of screening failure': 'SF_or_EOS_date'})

    # 處理DS
    ds_data = ds[['Screening Number', 'Date of completion/ Early Termination/ Withdrawal']].copy() if not ds.empty else pd.DataFrame(columns=['Screening Number', 'Date of completion/ Early Termination/ Withdrawal'])
    ds_data['exit_reason'] = 'EOS'
    ds_data = ds_data.rename(columns={'Date of completion/ Early Termination/ Withdrawal': 'SF_or_EOS_date'})

    # 合併SF和DS
    crf_combined = pd.concat([sf_data, ds_data], ignore_index=True)
    print(f"✓ CRF 數據：{len(crf_combined)} 筆（SF: {len(sf_data)}, EOS: {len(ds_data)}）")

    # 處理SV數據
    v1_data, last_visit_data = process_sv_data(sv)

    # 合併所有數據
    result = epro_latest[['Screening Number', 'Status', 'epro_date']].copy()
    result = result.rename(columns={'epro_date': 'epro_latest_date'})

    # 合併CRF數據
    result = result.merge(crf_combined, on='Screening Number', how='left')

    # 合併Expired日期
    result = result.merge(epro_expired, on='Screening Number', how='left')

    # 合併Normal日期（開帳日期）
    result = result.merge(epro_normal, on='Screening Number', how='left')

    # 合併V1數據
    result = result.merge(v1_data, on='Screening Number', how='left')

    # 合併最後visit數據
    result = result.merge(last_visit_data, on='Screening Number', how='left')

    # 合併試驗狀態
    if not subject_status.empty:
        result = result.merge(subject_status, on='Screening Number', how='left')
        print(f"✓ 試驗狀態已整合")
    else:
        result['試驗狀態'] = None

    # 處理 missing 值 - 將SF_or_EOS_date的NaN替換為'尚未填日期'
    result['SF_or_EOS_date'] = result['SF_or_EOS_date'].fillna('尚未填日期')

    # 排除指定的受試者
    before_exclude = len(result)
    result = result[~result['Screening Number'].isin(EXCLUDE_SUBJECTS)]
    after_exclude = len(result)
    if before_exclude > after_exclude:
        print(f"✓ 已排除 {before_exclude - after_exclude} 位受試者：{', '.join(EXCLUDE_SUBJECTS)}")

    print(f"✓ 合併完成：{len(result)} 位受試者")

    return result


def calculate_metrics(df):
    """計算各種指標"""
    print("\n" + "="*80)
    print("📊 步驟 5：計算指標")
    print("="*80)

    # 將'尚未填日期'替換為NaT以便計算
    df['SF_or_EOS_date_dt'] = df['SF_or_EOS_date'].replace('尚未填日期', pd.NaT)
    df['SF_or_EOS_date_dt'] = pd.to_datetime(df['SF_or_EOS_date_dt'])

    # 1. 計算關帳延遲天數
    df['days_to_closure'] = np.nan
    has_exit = df['SF_or_EOS_date_dt'].notna()
    is_expired = df['Status'] == 'Expired'
    has_expired_date = df['epro_expired_date'].notna()

    valid_closure = has_exit & is_expired & has_expired_date
    if valid_closure.any():
        df.loc[valid_closure, 'days_to_closure'] = (
            pd.to_datetime(df.loc[valid_closure, 'epro_expired_date']) -
            df.loc[valid_closure, 'SF_or_EOS_date_dt']
        ).dt.days

    # 2. 關帳狀態（已不需要，但保留計算以防程式其他地方使用）
    df['closure_status'] = 'Unknown'
    df.loc[has_exit & is_expired, 'closure_status'] = 'Closed'
    df.loc[has_exit & (df['Status'] == 'Normal'), 'closure_status'] = 'Not Closed'

    # 3. V1到開帳日期的天數
    df['days_V1_to_epro'] = np.nan
    has_v1 = df['V1_date'].notna()
    has_normal = df['epro_normal_date'].notna()
    if (has_v1 & has_normal).any():
        df.loc[has_v1 & has_normal, 'days_V1_to_epro'] = (
            pd.to_datetime(df.loc[has_v1 & has_normal, 'epro_normal_date']) -
            pd.to_datetime(df.loc[has_v1 & has_normal, 'V1_date'])
        ).dt.days

    # 4. 距離今天的天數（針對最後visit）
    df['days_since_last_visit'] = np.nan
    has_last = df['last_visit_date'].notna()
    if has_last.any():
        df.loc[has_last, 'days_since_last_visit'] = (
            TODAY - pd.to_datetime(df.loc[has_last, 'last_visit_date'])
        ).dt.days

    print("✓ 指標計算完成")
    print(f"   - 有SF/EOS日期：{has_exit.sum()} 位")
    print(f"   - 已關帳：{(df['closure_status'] == 'Closed').sum()} 位")
    print(f"   - 未關帳：{(df['closure_status'] == 'Not Closed').sum()} 位")

    return df


def identify_issues(df):
    """識別問題"""
    print("\n" + "="*80)
    print("🔍 步驟 6：識別問題")
    print("="*80)

    # 問題1：未關帳清單(有填EOS/SF頁)
    print("\n🔴 問題 1：未關帳清單(有填EOS/SF頁)")
    print("-"*80)
    not_closed = df[
        (df['SF_or_EOS_date'] != '尚未填日期') &
        (df['Status'] == 'Normal')
    ].copy()

    if len(not_closed) > 0:
        print(f"   ⚠️  {len(not_closed)} 位未關帳")
        not_closed['days_since_exit'] = (
            TODAY - pd.to_datetime(not_closed['SF_or_EOS_date_dt'])
        ).dt.days
        print(f"      最久：{not_closed['days_since_exit'].max():.0f} 天")
        print(f"\n   需立即處理：")
        for _, row in not_closed.sort_values('SF_or_EOS_date_dt').iterrows():
            date_str = row['SF_or_EOS_date'] if row['SF_or_EOS_date'] != '尚未填日期' else '尚未填日期'
            print(f"      Screening Number {row['Screening Number']}: {row['exit_reason']} 於 {date_str} "
                  f"(已 {int(row['days_since_exit'])} 天)")
    else:
        print(f"   ✓ 全部已關帳")

    # 問題2：V9完成但未關帳 (沒填EOS頁)
    print("\n🔴 問題 2：V9完成但未關帳 (沒填EOS頁)")
    print("-"*80)
    v9_not_closed = df[
        (df['試驗狀態'].str.contains('Ongoing', case=False, na=False)) &
        (df['last_visit'].notna()) &
        (df['last_visit'].str.contains('V9|EOS', case=False, na=False)) &
        (~df['last_visit'].str.contains('V6', case=False, na=False)) &  # 排除 V6
        (df['Status'] == 'Normal')
    ].copy()

    if len(v9_not_closed) > 0:
        print(f"   ⚠️  {len(v9_not_closed)} 位已完成V9但未關帳")
        print(f"\n   詳細列表：")
        for _, row in v9_not_closed.iterrows():
            print(f"      Screening Number {row['Screening Number']}: {row['last_visit']} 完成於 {row['last_visit_date']}, "
                  f"距今 {int(row['days_since_last_visit'])} 天")
    else:
        print(f"   ✓ 無此問題")

    # 問題3：V1_V2超過30天未關帳號 (沒填SF頁)
    print(f"\n🔴 問題 3：V1_V2超過{CUTOFF_DAYS}天未關帳號 (沒填SF頁)")
    print("-"*80)
    v1v2_not_closed = df[
        (df['last_visit'].notna()) &
        (df['last_visit'].str.contains('^V1|^V2', case=False, na=False, regex=True)) &
        (df['days_since_last_visit'] > CUTOFF_DAYS) &
        (df['SF_or_EOS_date'] == '尚未填日期') &
        (df['Status'] == 'Normal')  # 只顯示未關帳的
    ].copy()

    if len(v1v2_not_closed) > 0:
        print(f"   ⚠️  {len(v1v2_not_closed)} 位只到V1/V2超過{CUTOFF_DAYS}天且未關ePRO帳號")
        print(f"\n   詳細列表：")
        for _, row in v1v2_not_closed.iterrows():
            print(f"      Screening Number {row['Screening Number']}: {row['last_visit']} 於 {row['last_visit_date']}, "
                  f"距今 {int(row['days_since_last_visit'])} 天")
    else:
        print(f"   ✓ 無此問題")

    return not_closed, v9_not_closed, v1v2_not_closed


def auto_adjust_column_width(worksheet, df):
    """自動調整欄位寬度"""
    for idx, col in enumerate(df.columns):
        # 計算欄位名稱的長度
        column_len = len(str(col))

        # 計算該欄位中最長數據的長度
        if len(df) > 0:
            max_len = df[col].astype(str).str.len().max()
            column_len = max(column_len, max_len)

        # 設定寬度，最小8，最大50
        # 中文字符需要更多空間，所以乘以1.2
        adjusted_width = min(max(column_len * 1.2 + 2, 8), 50)

        # 取得欄位字母
        col_letter = chr(65 + idx) if idx < 26 else chr(65 + idx // 26 - 1) + chr(65 + idx % 26)
        worksheet.column_dimensions[col_letter].width = adjusted_width


def export_results(df, not_closed, v9_not_closed, v1v2_not_closed, compliance_df,
                  bgtrt_issues, nrs_issues, bgtrt_extra_issues, nrs_extra_issues):
    """匯出結果到單一Excel檔案的多個工作表"""
    print("\n" + "="*80)
    print("💾 步驟 7：匯出結果")
    print("="*80)

    # 準備輸出欄位
    output_cols = [
        'Screening Number', '試驗狀態', 'exit_reason', 'SF_or_EOS_date', 'Status', 'epro_expired_date',
        'days_to_closure', 'V1_date', 'epro_normal_date',
        'days_V1_to_epro', 'last_visit', 'last_visit_date', 'days_since_last_visit'
    ]

    for col in output_cols:
        if col not in df.columns:
            df[col] = None

    # 格式化日期欄位（確保只顯示日期）
    date_cols = ['epro_expired_date', 'V1_date', 'epro_normal_date', 'last_visit_date']
    for col in date_cols:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: str(x) if pd.notna(x) else '')

    # 特別處理SF_or_EOS_date - 將datetime格式轉為日期字串，保留'尚未填日期'
    if 'SF_or_EOS_date_dt' in df.columns:
        df['SF_or_EOS_date'] = df.apply(
            lambda row: str(row['SF_or_EOS_date_dt'].date())
            if pd.notna(row['SF_or_EOS_date_dt']) and row['SF_or_EOS_date'] != '尚未填日期'
            else row['SF_or_EOS_date'],
            axis=1
        )

    # 處理 exit_reason 空值
    df['exit_reason'] = df['exit_reason'].fillna('沒填SF或EOS頁面')
    df.loc[df['exit_reason'].isna() | (df['exit_reason'] == ''), 'exit_reason'] = '沒填SF或EOS頁面'

    # 處理 Status 欄位 - 如果不是 Normal 或 Expired，就是 "沒有辦ePRO"
    df['Status'] = df['Status'].apply(
        lambda x: x if x in ['Normal', 'Expired'] else '沒有辦ePRO'
    )

    # 重新命名欄位為中文
    column_names = {
        'Screening Number': 'Screening Number',
        '試驗狀態': '試驗狀態',
        'exit_reason': '結束原因',
        'SF_or_EOS_date': 'SF或EOS日期',
        'Status': 'ePRO帳號狀態',
        'epro_expired_date': 'ePRO關閉日期',
        'days_to_closure': 'SF/EOS後幾天關ePRO',
        'V1_date': 'V1日期',
        'epro_normal_date': 'ePRO開啟日期',
        'days_V1_to_epro': 'V1到開帳天數',
        'last_visit': '最新visit',
        'last_visit_date': '最新visit日期',
        'days_since_last_visit': '上一次visit距今幾天'
    }

    # 創建單一Excel檔案，包含多個工作表
    import os
    output_dir = '/mnt/user-data/outputs' if os.path.exists('/mnt/user-data/outputs') else '.'
    output_filename = os.path.join(output_dir, 'ePRO分析結果_完整報告.xlsx')
    print(f"\n📄 產生單一Excel檔案：{output_filename}")

    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        # 獲取workbook對象以便創建樣式
        workbook = writer.book

        # 定義標題列樣式
        from openpyxl.styles import Font, Alignment, PatternFill, Border, Side

        header_font = Font(bold=True, size=11)
        header_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
        header_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
        thin_border = Border(
            left=Side(style='thin'),
            right=Side(style='thin'),
            top=Side(style='thin'),
            bottom=Side(style='thin')
        )

        # 數據列樣式
        data_alignment = Alignment(horizontal="left", vertical="center", wrap_text=True)

        # 工作表1：未關帳清單(有填EOS/SF頁)
        sheet_name_1 = '1_未關帳清單(有填EOS或SF頁)'
        if len(not_closed) > 0:
            print(f"   ✓ 工作表 1：未關帳清單(有填EOS/SF頁) ({len(not_closed)} 位)")
            nc_cols = ['Screening Number', '試驗狀態', 'exit_reason', 'SF_or_EOS_date', 'Status', 'days_since_exit']
            nc_names = {k: column_names.get(k, k) for k in nc_cols}
            nc_names['days_since_exit'] = 'SF/EOS後已經過幾天'
            nc_export = not_closed[nc_cols].sort_values('days_since_exit', ascending=False)
            nc_export = nc_export.rename(columns=nc_names)
            nc_export.to_excel(writer, sheet_name=sheet_name_1, index=False)

            # 取得worksheet並套用樣式
            worksheet = writer.sheets[sheet_name_1]

            # 套用標題列樣式
            for col_num, value in enumerate(nc_export.columns.values, 1):
                cell = worksheet.cell(row=1, column=col_num)
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment
                cell.border = thin_border

            # 套用數據列樣式和邊框
            for row_num in range(2, len(nc_export) + 2):
                for col_num in range(1, len(nc_export.columns) + 1):
                    cell = worksheet.cell(row=row_num, column=col_num)
                    cell.alignment = data_alignment
                    cell.border = thin_border

            # 自動調整欄位寬度
            auto_adjust_column_width(worksheet, nc_export)
        else:
            print("   ✓ 工作表 1：未關帳清單(有填EOS/SF頁) (無資料)")
            empty_df = pd.DataFrame({'說明': ['全部已關帳']})
            empty_df.to_excel(writer, sheet_name=sheet_name_1, index=False)

            worksheet = writer.sheets[sheet_name_1]
            cell = worksheet.cell(row=1, column=1)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
            worksheet.column_dimensions['A'].width = 20

        # 工作表2：V9完成但未關帳 (沒填EOS頁)
        sheet_name_2 = '2_V9完成但未關帳(沒填EOS頁)'
        if len(v9_not_closed) > 0:
            print(f"   ✓ 工作表 2：V9完成但未關帳 (沒填EOS頁) ({len(v9_not_closed)} 位)")
            v9_cols = ['Screening Number', '試驗狀態', 'last_visit', 'last_visit_date', 'days_since_last_visit', 'Status']
            v9_names = {k: column_names.get(k, k) for k in v9_cols}
            v9_export = v9_not_closed[v9_cols].sort_values('days_since_last_visit', ascending=False)
            v9_export = v9_export.rename(columns=v9_names)
            v9_export.to_excel(writer, sheet_name=sheet_name_2, index=False)

            worksheet = writer.sheets[sheet_name_2]

            # 套用標題列樣式
            for col_num, value in enumerate(v9_export.columns.values, 1):
                cell = worksheet.cell(row=1, column=col_num)
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment
                cell.border = thin_border

            # 套用數據列樣式和邊框
            for row_num in range(2, len(v9_export) + 2):
                for col_num in range(1, len(v9_export.columns) + 1):
                    cell = worksheet.cell(row=row_num, column=col_num)
                    cell.alignment = data_alignment
                    cell.border = thin_border

            auto_adjust_column_width(worksheet, v9_export)
        else:
            print("   ✓ 工作表 2：V9完成但未關帳 (沒填EOS頁) (無資料)")
            empty_df = pd.DataFrame({'說明': ['無此問題']})
            empty_df.to_excel(writer, sheet_name=sheet_name_2, index=False)

            worksheet = writer.sheets[sheet_name_2]
            cell = worksheet.cell(row=1, column=1)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
            worksheet.column_dimensions['A'].width = 20

        # 工作表3：V1_V2超過30天未關帳號 (沒填SF頁)
        sheet_name_3 = '3_V1V2超過30天未關帳號(沒填SF頁)'
        if len(v1v2_not_closed) > 0:
            print(f"   ✓ 工作表 3：V1_V2超過30天未關帳號 (沒填SF頁) ({len(v1v2_not_closed)} 位)")
            v1v2_cols = ['Screening Number', '試驗狀態', 'last_visit', 'last_visit_date', 'days_since_last_visit',
                         'Status', 'SF_or_EOS_date']
            v1v2_names = {k: column_names.get(k, k) for k in v1v2_cols}
            v1v2_names['SF_or_EOS_date'] = 'SF日期'  # 特別修改這個工作表的欄位名稱
            v1v2_export = v1v2_not_closed[v1v2_cols].sort_values('days_since_last_visit', ascending=False)
            v1v2_export = v1v2_export.rename(columns=v1v2_names)
            v1v2_export.to_excel(writer, sheet_name=sheet_name_3, index=False)

            worksheet = writer.sheets[sheet_name_3]

            # 套用標題列樣式
            for col_num, value in enumerate(v1v2_export.columns.values, 1):
                cell = worksheet.cell(row=1, column=col_num)
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment
                cell.border = thin_border

            # 套用數據列樣式和邊框
            for row_num in range(2, len(v1v2_export) + 2):
                for col_num in range(1, len(v1v2_export.columns) + 1):
                    cell = worksheet.cell(row=row_num, column=col_num)
                    cell.alignment = data_alignment
                    cell.border = thin_border

            auto_adjust_column_width(worksheet, v1v2_export)
        else:
            print("   ✓ 工作表 3：V1_V2超過30天未關帳號 (沒填SF頁) (無資料)")
            empty_df = pd.DataFrame({'說明': ['無此問題']})
            empty_df.to_excel(writer, sheet_name=sheet_name_3, index=False)

            worksheet = writer.sheets[sheet_name_3]
            cell = worksheet.cell(row=1, column=1)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
            worksheet.column_dimensions['A'].width = 20

        # 工作表4：完整分析
        sheet_name_9 = '4_SF或EOS後關帳延遲'
        print(f"   ✓ 工作表 4：SF或EOS後關帳延遲")

        # 從完整分析中篩選有 SF/EOS 且已關帳的記錄
        closure_analysis = df[
            (df['SF_or_EOS_date'] != '尚未填日期') &
            (df['Status'] == 'Expired') &
            (df['days_to_closure'].notna())
        ].copy()

        if len(closure_analysis) > 0:
            closure_cols = ['Screening Number', '試驗狀態', 'exit_reason', 'SF_or_EOS_date',
                          'epro_expired_date', 'days_to_closure']
            closure_export = closure_analysis[closure_cols].copy()

            # 標記是否延遲（例如超過7天）
            closure_export['是否延遲'] = closure_export['days_to_closure'].apply(
                lambda x: '是' if x > DELAY_THRESHOLD else '否'
            )

            closure_export = closure_export.rename(columns={
                'Screening Number': 'Screening Number',
                '試驗狀態': '試驗狀態',
                'exit_reason': '結束原因',
                'SF_or_EOS_date': 'SF或EOS日期',
                'epro_expired_date': 'ePRO關閉日期',
                'days_to_closure': 'SF/EOS後幾天關ePRO'
            })

            # 依延遲天數排序
            closure_export = closure_export.sort_values('SF/EOS後幾天關ePRO', ascending=False)

            closure_export.to_excel(writer, sheet_name=sheet_name_9, index=False)
            worksheet = writer.sheets[sheet_name_9]

            # 套用樣式
            for col_num, value in enumerate(closure_export.columns.values, 1):
                cell = worksheet.cell(row=1, column=col_num)
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment
                cell.border = thin_border

            for row_num in range(2, len(closure_export) + 2):
                for col_num in range(1, len(closure_export.columns) + 1):
                    cell = worksheet.cell(row=row_num, column=col_num)
                    cell.alignment = data_alignment
                    cell.border = thin_border

            auto_adjust_column_width(worksheet, closure_export)

            print(f"      - 已關帳受試者：{len(closure_export)} 位")
            print(f"      - 延遲關帳（>{DELAY_THRESHOLD}天）：{(closure_export['是否延遲'] == '是').sum()} 位")
        else:
            empty_df = pd.DataFrame({'說明': ['無已關帳記錄']})
            empty_df.to_excel(writer, sheet_name=sheet_name_9, index=False)

            worksheet = writer.sheets[sheet_name_9]
            cell = worksheet.cell(row=1, column=1)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
            worksheet.column_dimensions['A'].width = 20

        # 工作表10：V1後延遲開帳分析
        sheet_name_10 = '5_V1後開帳延遲'
        print(f"   ✓ 工作表 5：V1後開帳延遲")

        # 從完整分析中篩選有 V1 且有開帳記錄的
        opening_analysis = df[
            (df['V1_date'].notna()) &
            (df['epro_normal_date'].notna()) &
            (df['days_V1_to_epro'].notna())
        ].copy()

        if len(opening_analysis) > 0:
            opening_cols = ['Screening Number', '試驗狀態', 'V1_date', 'epro_normal_date', 'days_V1_to_epro']
            opening_export = opening_analysis[opening_cols].copy()

            # 標記是否延遲（例如超過0天，即V1當天或之前應該開帳）
            opening_export['是否延遲'] = opening_export['days_V1_to_epro'].apply(
                lambda x: '是' if x > 0 else '否'
            )

            # 判定開帳時間點
            opening_export['開帳時間點'] = opening_export['days_V1_to_epro'].apply(
                lambda x: 'V1前開帳' if x < 0 else ('V1當天開帳' if x == 0 else 'V1後開帳')
            )

            opening_export = opening_export.rename(columns={
                'Screening Number': 'Screening Number',
                '試驗狀態': '試驗狀態',
                'V1_date': 'V1日期',
                'epro_normal_date': 'ePRO開啟日期',
                'days_V1_to_epro': 'V1到開帳天數'
            })

            # 依延遲天數排序
            opening_export = opening_export.sort_values('V1到開帳天數', ascending=False)

            opening_export.to_excel(writer, sheet_name=sheet_name_10, index=False)
            worksheet = writer.sheets[sheet_name_10]

            # 套用樣式
            for col_num, value in enumerate(opening_export.columns.values, 1):
                cell = worksheet.cell(row=1, column=col_num)
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment
                cell.border = thin_border

            for row_num in range(2, len(opening_export) + 2):
                for col_num in range(1, len(opening_export.columns) + 1):
                    cell = worksheet.cell(row=row_num, column=col_num)
                    cell.alignment = data_alignment
                    cell.border = thin_border

            auto_adjust_column_width(worksheet, opening_export)

            print(f"      - 有V1和開帳記錄：{len(opening_export)} 位")
            print(f"      - V1後才開帳：{(opening_export['是否延遲'] == '是').sum()} 位")
            print(f"      - V1當天開帳：{(opening_export['開帳時間點'] == 'V1當天開帳').sum()} 位")
            print(f"      - V1前開帳：{(opening_export['開帳時間點'] == 'V1前開帳').sum()} 位")
        else:
            empty_df = pd.DataFrame({'說明': ['無V1和開帳記錄']})
            empty_df.to_excel(writer, sheet_name=sheet_name_10, index=False)

            worksheet = writer.sheets[sheet_name_10]
            cell = worksheet.cell(row=1, column=1)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
            worksheet.column_dimensions['A'].width = 20



        sheet_name_4 = '6_ePRO帳號完整分析'
        print(f"   ✓ 工作表 6：ePRO帳號完整分析")
        full = df[output_cols].copy()
        full = full.rename(columns=column_names)
        full.to_excel(writer, sheet_name=sheet_name_4, index=False)

        worksheet = writer.sheets[sheet_name_4]

        # 套用標題列樣式
        for col_num, value in enumerate(full.columns.values, 1):
            cell = worksheet.cell(row=1, column=col_num)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border

        # 套用數據列樣式和邊框
        for row_num in range(2, len(full) + 2):
            for col_num in range(1, len(full.columns) + 1):
                cell = worksheet.cell(row=row_num, column=col_num)
                cell.alignment = data_alignment
                cell.border = thin_border

        auto_adjust_column_width(worksheet, full)

        # 工作表5：問卷填寫完整分析
        sheet_name_6 = '7_BGTRT問卷缺漏詳情'
        if len(bgtrt_issues) > 0:
            print(f"   ✓ 工作表 7：BGTRT問卷缺漏詳情 ({len(bgtrt_issues)} 位)")

            bgtrt_detail_cols = ['Screening Number', 'First visit date', 'Last visit', 'Last visit date',
                               'Expected end date', 'Expected days', 'BGTRT filled days',
                               'BGTRT missing days', 'BGTRT missing dates']
            bgtrt_detail = bgtrt_issues[bgtrt_detail_cols].copy()

            # 將缺漏日期列表轉換為字串（顯示完整日期）
            bgtrt_detail['BGTRT missing dates'] = bgtrt_detail['BGTRT missing dates'].apply(
                lambda x: ', '.join([str(d) for d in x]) if x else ''
            )

            bgtrt_detail = bgtrt_detail.rename(columns={
                'Screening Number': 'Screening Number',
                'First visit date': '首次訪視日期',
                'Last visit': '最後訪視',
                'Last visit date': '最後訪視日期',
                'Expected end date': '應填至日期',
                'Expected days': '應填天數',
                'BGTRT filled days': '已填天數',
                'BGTRT missing days': '缺漏天數',
                'BGTRT missing dates': '完整缺漏日期'
            })

            bgtrt_detail.to_excel(writer, sheet_name=sheet_name_6, index=False)
            worksheet = writer.sheets[sheet_name_6]

            for col_num, value in enumerate(bgtrt_detail.columns.values, 1):
                cell = worksheet.cell(row=1, column=col_num)
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment
                cell.border = thin_border

            for row_num in range(2, len(bgtrt_detail) + 2):
                for col_num in range(1, len(bgtrt_detail.columns) + 1):
                    cell = worksheet.cell(row=row_num, column=col_num)
                    cell.alignment = data_alignment
                    cell.border = thin_border

            auto_adjust_column_width(worksheet, bgtrt_detail)
        else:
            print("   ✓ 工作表 7：BGTRT問卷缺漏詳情 (無缺漏)")
            empty_df = pd.DataFrame({'說明': ['所有受試者BGTRT問卷皆完整填寫']})
            empty_df.to_excel(writer, sheet_name=sheet_name_6, index=False)

            worksheet = writer.sheets[sheet_name_6]
            cell = worksheet.cell(row=1, column=1)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
            worksheet.column_dimensions['A'].width = 40

        # 工作表7：NRS缺漏詳情
        sheet_name_7 = '8_NRS問卷缺漏詳情'
        if len(nrs_issues) > 0:
            print(f"   ✓ 工作表 8：NRS問卷缺漏詳情 ({len(nrs_issues)} 位)")

            nrs_detail_cols = ['Screening Number', 'First visit date', 'Last visit', 'Last visit date',
                              'Expected end date', 'Expected days', 'NRS filled days',
                              'NRS missing days', 'NRS missing dates']
            nrs_detail = nrs_issues[nrs_detail_cols].copy()

            # 將缺漏日期列表轉換為字串（顯示完整日期）
            nrs_detail['NRS missing dates'] = nrs_detail['NRS missing dates'].apply(
                lambda x: ', '.join([str(d) for d in x]) if x else ''
            )

            nrs_detail = nrs_detail.rename(columns={
                'Screening Number': 'Screening Number',
                'First visit date': '首次訪視日期',
                'Last visit': '最後訪視',
                'Last visit date': '最後訪視日期',
                'Expected end date': '應填至日期',
                'Expected days': '應填天數',
                'NRS filled days': '已填天數',
                'NRS missing days': '缺漏天數',
                'NRS missing dates': '完整缺漏日期'
            })

            nrs_detail.to_excel(writer, sheet_name=sheet_name_7, index=False)
            worksheet = writer.sheets[sheet_name_7]

            for col_num, value in enumerate(nrs_detail.columns.values, 1):
                cell = worksheet.cell(row=1, column=col_num)
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment
                cell.border = thin_border

            for row_num in range(2, len(nrs_detail) + 2):
                for col_num in range(1, len(nrs_detail.columns) + 1):
                    cell = worksheet.cell(row=row_num, column=col_num)
                    cell.alignment = data_alignment
                    cell.border = thin_border

            auto_adjust_column_width(worksheet, nrs_detail)
        else:
            print("   ✓ 工作表 8：NRS問卷缺漏詳情 (無缺漏)")
            empty_df = pd.DataFrame({'說明': ['所有受試者NRS問卷皆完整填寫']})
            empty_df.to_excel(writer, sheet_name=sheet_name_7, index=False)

            worksheet = writer.sheets[sheet_name_7]
            cell = worksheet.cell(row=1, column=1)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
            worksheet.column_dimensions['A'].width = 40

        # 工作表8：問卷多填詳情
        sheet_name_8 = '9_問卷多填詳情'

        # 準備多填資料，加入更多欄位
        bgtrt_extra_detail = bgtrt_extra_issues[['Screening Number', 'First visit date', 'Last visit',
                                                  'Last visit date', 'Expected end date',
                                                  'BGTRT extra days', 'BGTRT extra dates']].copy()
        bgtrt_extra_detail['問卷類型'] = 'BGTRT'
        bgtrt_extra_detail = bgtrt_extra_detail.rename(columns={
            'BGTRT extra days': '多填天數',
            'BGTRT extra dates': '多填日期列表'
        })

        nrs_extra_detail = nrs_extra_issues[['Screening Number', 'First visit date', 'Last visit',
                                             'Last visit date', 'Expected end date',
                                             'NRS extra days', 'NRS extra dates']].copy()
        nrs_extra_detail['問卷類型'] = 'NRS'
        nrs_extra_detail = nrs_extra_detail.rename(columns={
            'NRS extra days': '多填天數',
            'NRS extra dates': '多填日期列表'
        })

        extra_issues_combined = pd.concat([bgtrt_extra_detail, nrs_extra_detail], ignore_index=True)

        if len(extra_issues_combined) > 0:
            print(f"   ✓ 工作表 9：問卷多填詳情 ({len(extra_issues_combined)} 筆)")

            # 將多填日期列表轉換為字串（顯示完整日期）
            extra_issues_combined['完整多填日期'] = extra_issues_combined['多填日期列表'].apply(
                lambda x: ', '.join([str(d) for d in x]) if isinstance(x, list) else str(x)
            )

            # 重新整理欄位
            extra_detail = pd.DataFrame({
                'Screening Number': extra_issues_combined['Screening Number'],
                '首次訪視日期': extra_issues_combined['First visit date'],
                '最後訪視': extra_issues_combined['Last visit'],
                '最後訪視日期': extra_issues_combined['Last visit date'],
                '應填至日期': extra_issues_combined['Expected end date'],
                '問卷類型': extra_issues_combined['問卷類型'],
                '多填天數': extra_issues_combined['多填天數'],
                '完整多填日期': extra_issues_combined['完整多填日期']
            })

            extra_detail.to_excel(writer, sheet_name=sheet_name_8, index=False)
            worksheet = writer.sheets[sheet_name_8]

            for col_num, value in enumerate(extra_detail.columns.values, 1):
                cell = worksheet.cell(row=1, column=col_num)
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment
                cell.border = thin_border

            for row_num in range(2, len(extra_detail) + 2):
                for col_num in range(1, len(extra_detail.columns) + 1):
                    cell = worksheet.cell(row=row_num, column=col_num)
                    cell.alignment = data_alignment
                    cell.border = thin_border

            auto_adjust_column_width(worksheet, extra_detail)
        else:
            print("   ✓ 工作表 9：問卷多填詳情 (無多填)")
            empty_df = pd.DataFrame({'說明': ['無受試者有問卷多填情況']})
            empty_df.to_excel(writer, sheet_name=sheet_name_8, index=False)

            worksheet = writer.sheets[sheet_name_8]
            cell = worksheet.cell(row=1, column=1)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
            worksheet.column_dimensions['A'].width = 40

        # 工作表9：SF/EOS後延遲關閉ePRO分析
        sheet_name_5 = '10_問卷填寫完整分析'
        print(f"   ✓ 工作表 10：問卷填寫完整分析 ({len(compliance_df)} 位)")

        # 準備輸出欄位（加入最後訪視日期）
        quest_cols = ['Screening Number', 'First visit date', 'Last visit', 'Last visit date',
                     'Expected end date', 'Expected days',
                     'BGTRT filled days', 'BGTRT missing days', 'BGTRT compliance',
                     'NRS filled days', 'NRS missing days', 'NRS compliance']

        quest_export = compliance_df[quest_cols].copy()
        quest_export = quest_export.rename(columns={
            'Screening Number': 'Screening Number',
            'First visit date': '首次訪視日期',
            'Last visit': '最後訪視',
            'Last visit date': '最後訪視日期',
            'Expected end date': '應填至日期',
            'Expected days': '應填天數',
            'BGTRT filled days': 'BGTRT已填天數',
            'BGTRT missing days': 'BGTRT缺漏天數',
            'BGTRT compliance': 'BGTRT完成率',
            'NRS filled days': 'NRS已填天數',
            'NRS missing days': 'NRS缺漏天數',
            'NRS compliance': 'NRS完成率'
        })

        quest_export.to_excel(writer, sheet_name=sheet_name_5, index=False)
        worksheet = writer.sheets[sheet_name_5]

        # 套用樣式
        for col_num, value in enumerate(quest_export.columns.values, 1):
            cell = worksheet.cell(row=1, column=col_num)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border

        for row_num in range(2, len(quest_export) + 2):
            for col_num in range(1, len(quest_export.columns) + 1):
                cell = worksheet.cell(row=row_num, column=col_num)
                cell.alignment = data_alignment
                cell.border = thin_border

        auto_adjust_column_width(worksheet, quest_export)

    print(f"\n✅ 完成！所有結果已整合到：{output_filename}")

    # 在 Colab 環境中自動下載
    try:
        from google.colab import files
        print("\n📥 開始下載檔案...")
        files.download(output_filename)
        print("✓ 檔案下載完成")
    except:
        print("\n💡 提示：檔案已產生，請從檔案列表手動下載")

        # 工作表6：BGTRT缺漏詳情
def generate_summary(df, not_closed, v9_not_closed, v1v2_not_closed, compliance_df,
                     bgtrt_issues, nrs_issues, bgtrt_extra_issues, nrs_extra_issues):
    """產生摘要"""
    print("\n" + "="*80)
    print("📈 分析摘要")
    print("="*80)

    total = len(df[df['SF_or_EOS_date'] != '尚未填日期'])
    print(f"\n📊 基本統計：")
    print(f"   總受試者：{len(df)} 位")
    print(f"   已 SF/EOS：{total} 位")

    if total > 0:
        sf = len(df[df['exit_reason'] == 'SF'])
        eos = len(df[df['exit_reason'] == 'EOS'])
        print(f"\n   結束原因：")
        print(f"      SF：{sf} 位 ({sf/total*100:.1f}%)")
        print(f"      EOS：{eos} 位 ({eos/total*100:.1f}%)")

        closed = len(df[(df['SF_or_EOS_date'] != '尚未填日期') & (df['Status'] == 'Expired')])
        print(f"\n   關帳狀況：")
        print(f"      已關閉：{closed} 位 ({closed/total*100:.1f}%)")
        print(f"      未關閉：{len(not_closed)} 位 ({len(not_closed)/total*100:.1f}%)")

    print(f"\n🚨 需處理問題：")
    print(f"   未關帳清單(有填EOS/SF頁)：{len(not_closed)} 位")
    print(f"   V9完成但未關帳 (沒填EOS頁)：{len(v9_not_closed)} 位")
    print(f"   V1/V2超過30天未關帳號 (沒填SF頁)：{len(v1v2_not_closed)} 位")

    print(f"\n📋 問卷填寫情況：")
    print(f"   Background Treatment 有缺漏：{len(bgtrt_issues)} 位")
    print(f"   NRS 有缺漏：{len(nrs_issues)} 位")
    print(f"   Background Treatment 有多填：{len(bgtrt_extra_issues)} 位")
    print(f"   NRS 有多填：{len(nrs_extra_issues)} 位")

    # 計算平均完成率
    avg_bgtrt = compliance_df['BGTRT filled days'].sum() / compliance_df['Expected days'].sum() * 100 if compliance_df['Expected days'].sum() > 0 else 0
    avg_nrs = compliance_df['NRS filled days'].sum() / compliance_df['Expected days'].sum() * 100 if compliance_df['Expected days'].sum() > 0 else 0
    print(f"\n   平均問卷完成率：")
    print(f"      Background Treatment：{avg_bgtrt:.1f}%")
    print(f"      NRS：{avg_nrs:.1f}%")


def main():
    """主程式"""

    # 在 Colab 環境上傳檔案
    try:
        from google.colab import files
        print("\n" + "="*80)
        print("📤 請上傳檔案")
        print("="*80)
        print("請上傳以下檔案：")
        print("1. crf_data.xlsx（包含 SF、DS、SV 工作表）")
        print("2. epro_data.xlsx（包含 epro 工作表）")
        print("3. subject_list.xlsx（包含 Screening No. 和 Status 欄位）")
        print("4. epro_content_data.xlsx（包含 BGTRT1、BGTRT2、NRS1、NRS2 工作表）")
        print()

        uploaded = files.upload()

        # 找出上傳的檔案
        crf_file = None
        epro_file = None
        subject_list_file = None
        content_file = None

        for filename in uploaded.keys():
            if 'crf' in filename.lower() and 'content' not in filename.lower():
                crf_file = filename
                print(f"✓ 找到 CRF 檔案：{filename}")
            elif 'epro' in filename.lower() and 'content' not in filename.lower():
                epro_file = filename
                print(f"✓ 找到 ePRO 檔案：{filename}")
            elif 'subject' in filename.lower() or 'list' in filename.lower():
                subject_list_file = filename
                print(f"✓ 找到 Subject List 檔案：{filename}")
            elif 'content' in filename.lower():
                content_file = filename
                print(f"✓ 找到 ePRO Content 檔案：{filename}")

        if not crf_file or not epro_file or not content_file:
            print("\n❌ 錯誤：請確認已上傳所有必要檔案")
            return

    except ImportError:
        # 不在 Colab 環境，使用預設路徑
        print("\n⚠️ 非 Colab 環境，使用預設檔案路徑")
        crf_file = 'crf_data.xlsx'
        epro_file = 'epro_data.xlsx'
        subject_list_file = 'subject_list.xlsx'
        content_file = 'epro_content_data.xlsx'

    # 讀取 ePRO 問卷填寫內容
    bgtrt_all, nrs_all, bgtrt_summary, nrs_summary = read_epro_content_data(content_file)

    # 讀取 CRF 數據（需要在分析問卷合規性之前讀取）
    sf, ds, sv = read_crf_data(crf_file)

    # 讀取 subject_list（試驗狀態）
    if subject_list_file:
        subject_status, subject_list_full = read_subject_list(subject_list_file)
    else:
        print("\n⚠️ 未提供 Subject List 檔案，試驗狀態將顯示為空白")
        subject_status = pd.DataFrame(columns=['Screening Number', '試驗狀態'])
        subject_list_full = pd.DataFrame()

    # 分析問卷填寫合規性（需要 sf 和 sv 資料）
    if not subject_list_full.empty:
        compliance_df, bgtrt_issues, nrs_issues, bgtrt_extra_issues, nrs_extra_issues = \
            analyze_questionnaire_compliance(bgtrt_summary, nrs_summary, subject_list_full, sf, sv)
    else:
        print("\n⚠️ 無法分析問卷合規性（缺少 Subject List）")
        compliance_df = pd.DataFrame()
        bgtrt_issues = pd.DataFrame()
        nrs_issues = pd.DataFrame()
        bgtrt_extra_issues = pd.DataFrame()
        nrs_extra_issues = pd.DataFrame()

    # 讀取 ePRO 數據
    epro_latest, epro_expired, epro_normal = read_epro_data(epro_file)

    if len(sf) == 0 and len(ds) == 0:
        print("\n❌ 無 CRF 數據")
        return

    if len(epro_latest) == 0:
        print("\n❌ 無 ePRO 數據")
        return

    # 合併和計算
    merged = merge_all_data(sf, ds, sv, epro_latest, epro_expired, epro_normal, subject_status)
    result = calculate_metrics(merged)

    # 識別問題
    not_closed, v9_not_closed, v1v2_not_closed = identify_issues(result)

    # 匯出結果
    export_results(result, not_closed, v9_not_closed, v1v2_not_closed,
                  compliance_df, bgtrt_issues, nrs_issues, bgtrt_extra_issues, nrs_extra_issues)

    # 摘要
    generate_summary(result, not_closed, v9_not_closed, v1v2_not_closed,
                    compliance_df, bgtrt_issues, nrs_issues, bgtrt_extra_issues, nrs_extra_issues)

    print("\n" + "="*80)
    print("✅ 完成！")
    print("="*80)


if __name__ == "__main__":
    main()
